In [3]:
pip install simpletransformers 

     |████████████████████████████████| 248 kB 17.6 MB/s 
     |████████████████████████████████| 325 kB 64.1 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 9.9 MB 53.9 MB/s 
     |████████████████████████████████| 6.5 MB 59.0 MB/s 
     |████████████████████████████████| 1.7 MB 54.4 MB/s 
     |████████████████████████████████| 1.2 MB 65.5 MB/s 
     |████████████████████████████████| 3.8 MB 62.3 MB/s 
     |████████████████████████████████| 895 kB 64.2 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 596 kB 65.0 MB/s 
     |████████████████████████████████| 144 kB 77.2 MB/s 
     |████████████████████████████████| 181 kB 76.1 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 
     |████████████████████████████████| 1.1 MB 59.2 MB/s 
     |████████████████████████████████| 134 kB 79.6 MB/s 
     |██████████████

In [4]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args
import torch

In [5]:
# Read in train and test data

def read_in_data():
    X_train = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/x_train_sampled_yelp_data.csv")
    y_train = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/y_train_sampled_yelp_data.csv")
    X_test = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/x_test_sampled_yelp_data.csv")
    y_test = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/y_test_sampled_yelp_data.csv")
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = read_in_data()

In [10]:
max_train_rows = 1000
train_data = pd.DataFrame(X_train[:max_train_rows]['text'])
train_data['target_text'] = y_train[:max_train_rows]['stars'].astype(str)
train_data.insert(0, 'prefix', 'predict sentiment')
train_data.columns = ["prefix", "input_text", "target_text"]
train_data

,prefix,input_text,target_text
0,predict sentiment,Sunday afternoon savior. They are great! Helpe...,5.0
1,predict sentiment,Tuesday was my man and I's weekly wing night. ...,4.0
2,predict sentiment,Everything is presented very nicely at Kishimo...,4.0
3,predict sentiment,So good we are thinking of having them cater o...,5.0
4,predict sentiment,Wow - pleasantly surprised!! Both my son and I...,4.0
...,...,...,...
995,predict sentiment,I like my coffee with much less attitude. Even...,1.0
996,predict sentiment,"On the other hand, if you are going for Thai, ...",2.0
997,predict sentiment,Surprisingly really fun and educational! They ...,4.0
998,predict sentiment,I heard it wasn't good from people who usually...,1.0


In [11]:
max_test_rows = 1000
test_data = pd.DataFrame(X_test[:max_test_rows]['text'])
test_data['stars'] = y_test[:max_test_rows]['stars'].astype(str)
test_data.insert(0, 'prefix', 'predict sentiment')
test_data.columns = ["prefix", "input_text", "target_text"]
test_data

,prefix,input_text,target_text
0,predict sentiment,The food was delicious. The microbrews were go...,4.0
1,predict sentiment,We arrived for brunch on a Sunday right before...,5.0
2,predict sentiment,Great pizza! Nice service...always a good exp...,4.0
3,predict sentiment,I CAN FINALLY WRITE REVIEWS FOR CANADA!!!\n\nM...,4.0
4,predict sentiment,"Fun atmosphere. Busy, but had a good system an...",4.0
...,...,...,...
995,predict sentiment,This place is not clean. There are stains on t...,1.0
996,predict sentiment,Sooooo delicious! Friendly staff and the food ...,5.0
997,predict sentiment,The WORST pedicure I have ever gotten. I was ...,1.0
998,predict sentiment,This place is a must. Tim was so knowledgeable...,5.0


In [5]:
model_args = T5Args()
model_args.num_train_epochs = 1
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True

model = T5Model("t5", "t5-base", args=model_args,use_cuda=False)


def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

In [6]:
torch.multiprocessing.set_sharing_strategy('file_system')
#ulimit -n 64000
#loss is cross-entropy

In [7]:
# Trial with 1000 rows
model.train_model(train_data, eval_data=test_data, matches=count_matches)

print(model.eval_model(test_data, matches=count_matches))

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the 

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

['4.0', '5.0', '4.0', '4.0', '4.0', '5.0', '5.0', '1.0', '5.0', '3.0', '5.0', '2.0', '3.0', '5.0', '5.0', '5.0', '3.0', '2.0', '5.0', '4.0', '5.0', '5.0', '4.0', '4.0', '2.0', '2.0', '2.0', '4.0', '1.0', '3.0', '5.0', '4.0', '3.0', '5.0', '2.0', '4.0', '4.0', '4.0', '5.0', '2.0', '5.0', '4.0', '5.0', '5.0', '5.0', '3.0', '3.0', '4.0', '1.0', '2.0', '1.0', '5.0', '4.0', '5.0', '1.0', '5.0', '5.0', '1.0', '5.0', '5.0', '5.0', '3.0', '5.0', '1.0', '4.0', '3.0', '5.0', '1.0', '5.0', '4.0', '5.0', '5.0', '3.0', '4.0', '3.0', '5.0', '5.0', '5.0', '5.0', '4.0', '1.0', '4.0', '4.0', '5.0', '1.0', '5.0', '3.0', '4.0', '5.0', '5.0', '4.0', '5.0', '1.0', '5.0', '5.0', '5.0', '4.0', '5.0', '5.0', '2.0', '1.0', '5.0', '5.0', '5.0', '1.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '3.0', '4.0', '1.0', '1.0', '5.0', '1.0', '5.0', '5.0', '5.0', '1.0', '5.0', '1.0', '5.0', '5.0', '4.0', '5.0', '1.0', '1.0', '2.0', '1.0', '5.0', '1.0', '5.0', '1.0', '2.0', '3.0', '5.0', '5.0', '2.0', '5.0', '5.0'

In [6]:
max_train_rows = 80000
train_data = pd.DataFrame(X_train[:max_train_rows]['text'])
train_data['target_text'] = y_train[:max_train_rows]['stars'].astype(str)
train_data.insert(0, 'prefix', 'predict sentiment')
train_data.columns = ["prefix", "input_text", "target_text"]
#train_data

In [7]:
max_test_rows = 20000
test_data = pd.DataFrame(X_test[:max_test_rows]['text'])
test_data['stars'] = y_test[:max_test_rows]['stars'].astype(str)
test_data.insert(0, 'prefix', 'predict sentiment')
test_data.columns = ["prefix", "input_text", "target_text"]
#test_data

In [8]:
model_args = T5Args()
model_args.num_train_epochs = 1
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True

model = T5Model("t5", "t5-base", args=model_args,use_cuda=False)


def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

In [11]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

In [ ]:
# Trial with all rows
model.train_model(train_data, eval_data=test_data, matches=count_matches,use_gpu = True)


  0%|          | 0/80000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
print(model.eval_model(test_data, matches=count_matches))

  0%|          | 0/20000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Running Evaluation:   0%|          | 0/2500 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/2500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
